In [2]:
import os
import math
import torch
import torch.nn as nn
from pathlib import Path

from torch.utils.data import Dataset, DataLoader, random_split

# Huggingface datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm

import torchmetrics
from torch.utils.tensorboard import SummaryWriter

In [8]:
# Transformer model parameters
MODEL_NUMBER_OF_LAYERS = 3
MODEL_DIMENSION = 256
MODEL_NUMBER_OF_HEADS = 4
MODEL_INNER_LAYER_DIMENSION = 1024
MODEL_DROPOUT_PROBABILITY = 0.1
MODEL_LABEL_SMOOTHING_VALUE = 0.1
SEQUENCE_LENGTH = 500

# Training parameters
BATCH_SIZE = 4
NUMBER_OF_EPOCHS = 10
BETA1 = 0.9
BETA2 = 0.98
EPSILON = 1e-9
WARMUP_STEPS = 4000

# Dataset parameters
DATASET_NAME = "Helsinki-NLP/opus_books"
SOURCE_LANGUAGE = "en"
TARGET_LANGUAGE = "fr"

# Saving parameters
MODEL_FOLDER = "weights"
MODEL_BASENAME = "tmodel_"
MODEL_PRELOAD = "latest"
EXPERIMENT_FOLDER = "runs/tmodel"

# Special tokens
UNK_TOKEN = '[UNK]'
SOS_TOKEN = '[SOS]'
EOS_TOKEN = '[EOS]'
PAD_TOKEN = "[PAD]"

CHECKPOINTS_PATH = os.path.join(os.getcwd(), 'models', 'checkpoints') # semi-trained models during training will be dumped here
BINARIES_PATH = os.path.join(os.getcwd(), 'models', 'binaries') # location where trained models are located
DATA_DIR_PATH = os.path.join(os.getcwd(), 'data') # training data will be stored here

os.makedirs(CHECKPOINTS_PATH, exist_ok=True)
os.makedirs(BINARIES_PATH, exist_ok=True)
os.makedirs(DATA_DIR_PATH, exist_ok=True)

def get_weights_file_path(epoch: str):
    model_folder = f"{DATASET_NAME}_{MODEL_FOLDER}"
    model_filename = f"{MODEL_BASENAME}{epoch}.pt"
    return str(Path('.') / model_folder / model_filename)

# Find the latest weights file in the weights folder
def latest_weights_file_path():
    model_folder = f"{DATASET_NAME}_{MODEL_FOLDER}"
    model_filename = f"{MODEL_BASENAME}*"
    weights_files = list(Path(model_folder).glob(model_filename))
    if len(weights_files) == 0:
        return None
    weights_files.sort()
    return str(weights_files[-1])

In [9]:
class Embedding(nn.Module):
    def __init__(self, vocabulary_size, model_dimension):
        super().__init__()
        self.vocabulary_size = vocabulary_size
        self.model_dimension = model_dimension
        self.embedding = nn.Embedding(vocabulary_size, model_dimension)

    def forward(self, src_token_ids):
        return self.embedding(src_token_ids) * math.sqrt(self.model_dimension)


class PositionalEncoding(nn.Module):
    def __init__(self, model_dimension, max_sequence_length=5000):
        super().__init__()
        self.sequence_length = max_sequence_length
        self.model_dimension = model_dimension

        positional_encoding = torch.zeros(max_sequence_length, model_dimension)

        positions = torch.arange(0, max_sequence_length, 1, dtype=float)
        positions = torch.unsqueeze(positions, 1)
        denominator = torch.exp(torch.arange(0, model_dimension, 2, dtype=float) * -math.log(10000.) / model_dimension)

        positional_encoding[:,0::2] = torch.sin(positions * denominator)
        positional_encoding[:,1::2] = torch.cos(positions * denominator)

        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer("positional_encoding", positional_encoding)

    def forward(self, src_embedded):
        return src_embedded + self.positional_encoding[:, :src_embedded.shape[1]]


class ScaledDotProductAttention(nn.Module):
    def __init__(self, key_dimension, value_dimension):
        super().__init__()
        self.key_dimension = key_dimension
        self.value_dimension = value_dimension

    def forward(self, queries, keys, values, mask=None):
        dot_product = torch.matmul(queries, keys.transpose(-2, -1))
        dot_product /= math.sqrt(self.key_dimension)

        if mask is not None:
            dot_product.masked_fill_(mask == 0, float("-inf"))

        weights = dot_product.softmax(dim=-1)

        return torch.matmul(weights, values), weights


class MultiHeadAttention(nn.Module):
    def __init__(self, model_dimension, number_of_heads, save_weigths=False):
        super().__init__()
        self.model_dimension = model_dimension
        self.number_of_heads = number_of_heads
        self.key_dimension = model_dimension // number_of_heads
        self.value_dimension = model_dimension // number_of_heads

        self.linear_queries = nn.Linear(model_dimension, model_dimension) # W_Q
        self.linear_keys = nn.Linear(model_dimension, model_dimension) # W_K
        self.linear_values = nn.Linear(model_dimension, model_dimension) # W_V

        self.linear_output = nn.Linear(model_dimension, model_dimension) # w_0

        self.attention_weigths = None
        self.save_weigths = save_weigths

    def forward(self, queries, keys, values, mask=None):
        projected_queries = self.linear_queries(queries) # Q * W_Q
        projected_queries = projected_queries.view(projected_queries.shape[0], projected_queries.shape[1], self.number_of_heads, self.key_dimension).transpose(1, 2)

        projected_keys = self.linear_keys(keys) # K * W_K
        projected_keys = projected_keys.view(projected_keys.shape[0], projected_keys.shape[1], self.number_of_heads, self.key_dimension).transpose(1, 2)

        projected_values = self.linear_values(values) # V * W_V
        projected_values = projected_values.view(projected_values.shape[0], projected_values.shape[1], self.number_of_heads, self.value_dimension).transpose(1, 2)

        scaled_dot_product = ScaledDotProductAttention(self.key_dimension, self.value_dimension)

        attention, attention_weigths = scaled_dot_product(projected_queries, projected_keys, projected_values, mask)
        attention = attention.transpose(1, 2).contiguous()
        attention = attention.view(queries.shape[0], -1, self.number_of_heads * self.key_dimension)

        if self.save_weigths :
            self.attention_weigths = attention_weigths.detach()

        return self.linear_output(attention)


class PositionwiseFeedForwardNetwork(nn.Module):
    def __init__(self, model_dimension, inner_layer_dimension):
        super().__init__()
        self.modul_dimension = model_dimension
        self.inner_layer_dimension = inner_layer_dimension

        self.linear1 = nn.Linear(model_dimension, inner_layer_dimension) # W1 and b1
        self.linear2 = nn.Linear(inner_layer_dimension, model_dimension) # W2 and b2
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        return self.linear2(x)


class EncoderLayer(nn.Module):
    def __init__(self, model_dimension, multihead_attention, feedforward_network):
        super().__init__()
        self.model_dimension = model_dimension

        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network

        self.layernorm1 = nn.LayerNorm(model_dimension)
        self.layernorm2 = nn.LayerNorm(model_dimension)

    def forward(self, src_embedded, src_mask=None):
        attention_ouput = self.multihead_attention(src_embedded, src_embedded, src_embedded, src_mask)
        sublayer_output1 = self.layernorm1(src_embedded + attention_ouput)

        ffnetwork_output = self.feedforward_network(sublayer_output1)
        sublayer_output2 = self.layernorm2(sublayer_output1 + ffnetwork_output)

        return sublayer_output2


class Encoder(nn.Module):
    def __init__(self, model_dimension, number_of_layers, number_of_heads, inner_layer_dimension):
        super().__init__()
        self.model_dimension = model_dimension
        self.number_of_layers = number_of_layers

        self.layers = nn.ModuleList([
            EncoderLayer(
                model_dimension,
                MultiHeadAttention(model_dimension, number_of_heads),
                PositionwiseFeedForwardNetwork(model_dimension, inner_layer_dimension)
            )
            for _ in range(number_of_layers)
        ])

    def forward(self, src_embedded, src_mask=None):
        src_encoder_output = src_embedded

        for layer in self.layers:
            src_encoder_output = layer(src_encoder_output, src_mask)

        return src_encoder_output


class DecoderLayer(nn.Module):
    def __init__(self, model_dimension, masked_multihead_attention, multihead_attention, feedforward_network):
        super().__init__()
        self.model_dimension = model_dimension

        self.masked_multihead_attention = masked_multihead_attention
        self.multihead_attention = multihead_attention
        self.feedforward_network = feedforward_network

        self.layernorm1 = nn.LayerNorm(model_dimension)
        self.layernorm2 = nn.LayerNorm(model_dimension)
        self.layernorm3 = nn.LayerNorm(model_dimension)

    def forward(self, src_encoder_output, trg_embedded, src_mask=None, trg_mask=None):
        masked_attention_output = self.masked_multihead_attention(queries=trg_embedded, keys=trg_embedded, values=trg_embedded, mask=trg_mask)
        sublayer_output1 = self.layernorm1(trg_embedded + masked_attention_output)

        attention_ouput = self.multihead_attention(queries=sublayer_output1, keys=src_encoder_output, values=src_encoder_output, mask=src_mask)
        sublayer_output2 = self.layernorm2(sublayer_output1 + attention_ouput)

        ffnetwork_ouput = self.feedforward_network(sublayer_output2)
        sublayer_output3 = self.layernorm3(sublayer_output2 + ffnetwork_ouput)

        return sublayer_output3


class Decoder(nn.Module):
    def __init__(self, model_dimension, number_of_layers, number_of_heads, inner_layer_dimension):
        super().__init__()
        self.model_dimension = model_dimension
        self.number_of_layers = number_of_layers

        self.layers = nn.ModuleList([
            DecoderLayer(
                model_dimension,
                MultiHeadAttention(model_dimension, number_of_heads),  # masked self-attention
                MultiHeadAttention(model_dimension, number_of_heads),  # encoder-decoder attention
                PositionwiseFeedForwardNetwork(model_dimension, inner_layer_dimension)
            )
            for _ in range(number_of_layers)
        ])

    def forward(self, src_encoder_output, trg_embedded, src_mask=None, trg_mask=None):
        trg_decoder_output = trg_embedded

        for layer in self.layers:
            trg_decoder_output = layer(src_encoder_output, trg_decoder_output, src_mask=src_mask, trg_mask=trg_mask)

        return trg_decoder_output


class Transformer(nn.Module):
    def __init__(self, model_dimension, inner_layer_dimension, number_of_layers, number_of_heads, src_vocabulary_size, trg_vocabulary_size):
        super().__init__()
        self.model_dimension = model_dimension
        self.number_of_layers = number_of_layers
        self.number_of_heads = number_of_heads
        self.src_vocabulary_size = src_vocabulary_size
        self.trg_vocabulary_size = trg_vocabulary_size

        self.input_embedding = Embedding(src_vocabulary_size, model_dimension)
        self.output_embedding = Embedding(trg_vocabulary_size, model_dimension)

        self.input_pos_encoding = PositionalEncoding(model_dimension)
        self.output_pos_encoding = PositionalEncoding(model_dimension)

        self.encoder = Encoder(model_dimension, number_of_layers, number_of_heads, inner_layer_dimension)
        self.decoder = Decoder(model_dimension, number_of_layers, number_of_heads, inner_layer_dimension)

        self.linear_projection = nn.Linear(model_dimension, trg_vocabulary_size)
        self.softmax = nn.Softmax(dim=-1)

    def encode(self, src_token_ids, src_mask=None):
        input_embedded = self.input_embedding(src_token_ids)
        input_pos_encoded = self.input_pos_encoding(input_embedded)
        input_encoded = self.encoder(input_pos_encoded, src_mask)
        return input_encoded

    def decode(self, input_encoded, trg_token_ids, src_mask, trg_mask=None):
        output_embedded = self.output_embedding(trg_token_ids)
        output_pos_encoded = self.output_pos_encoding(output_embedded)
        output_decoded = self.decoder(input_encoded, output_pos_encoded, src_mask, trg_mask)
        return output_decoded

    def forward(self, src_token_ids, trg_token_ids, src_mask=None, trg_mask=None):
        input_embedded = self.input_embedding(src_token_ids)
        input_pos_encoded = self.input_pos_encoding(input_embedded)
        input_encoded = self.encoder(input_pos_encoded, src_mask)

        output_embedded = self.output_embedding(trg_token_ids)
        output_pos_encoded = self.output_pos_encoding(output_embedded)
        output_decoded = self.decoder(input_encoded, output_pos_encoded, src_mask, trg_mask)

        output_decoded = self.linear_projection(output_decoded)
        # output_decoded = output_decoded.softmax(dim=-1)

        return output_decoded

In [10]:
class EnglishToFrenchDataset(Dataset):

    def __init__(self, dataset, src_tokenizer, trg_tokenizer, src_language, trg_language, sequence_length):
        super().__init__()
        self.sequence_length = sequence_length

        self.dataset = dataset
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer
        self.src_language = src_language
        self.trg_language = trg_language

        self.sos_token = torch.tensor([trg_tokenizer.token_to_id(SOS_TOKEN)], dtype=torch.int64)
        self.eos_token = torch.tensor([trg_tokenizer.token_to_id(EOS_TOKEN)], dtype=torch.int64)
        self.pad_token = torch.tensor([trg_tokenizer.token_to_id(PAD_TOKEN)], dtype=torch.int64)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        src_target_pair = self.dataset[idx]
        src_text = src_target_pair['translation'][self.src_language]
        trg_text = src_target_pair['translation'][self.trg_language]

        # Transform the text into tokens
        enc_input_tokens = self.src_tokenizer.encode(src_text).ids
        dec_input_tokens = self.trg_tokenizer.encode(trg_text).ids

        max_token_length = self.sequence_length - 2
        if len(enc_input_tokens) > max_token_length:
            enc_input_tokens = enc_input_tokens[:max_token_length]
        if len(dec_input_tokens) > max_token_length - 1:  # -1 car decoder a seulement SOS au début
            dec_input_tokens = dec_input_tokens[:max_token_length - 1]

        # Add sos, eos and padding to each sentence
        enc_num_padding_tokens = self.sequence_length - len(enc_input_tokens) - 2  # We will add <s> and </s>
        # We will only add <s>, and </s> only on the label
        dec_num_padding_tokens = self.sequence_length - len(dec_input_tokens) - 1

        # Make sure the number of padding tokens is not negative. If it is, the sentence is too long
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Add <s> and </s> token
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only <s> token
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Double check the size of the tensors to make sure they are all sequence_length long
        assert encoder_input.size(0) == self.sequence_length
        assert decoder_input.size(0) == self.sequence_length
        assert label.size(0) == self.sequence_length

        encoder_mask = (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0)
        decoder_mask = (decoder_input != self.pad_token).unsqueeze(0) & causal_mask(decoder_input.size(0))

        return {
            "encoder_input": encoder_input,  # (sequence_length)
            "decoder_input": decoder_input,  # (sequence_length)
            "encoder_mask": encoder_mask, # (1, 1, sequence_length)
            "decoder_mask": decoder_mask, # (1, sequence_length) & (1, sequence_length, sequence_length),
            "label": label,  # (sequence_length)
            "src_text": src_text,
            "trg_text": trg_text,
        }

def causal_mask(size):
    mask = torch.triu(torch.ones((size, size)), diagonal=1).type(torch.int)
    return (mask == 0).unsqueeze(0)


def get_all_sentences(dataset, language):
    for item in dataset:
        yield item["translation"][language]


def get_tokenizer(dataset, language):
    tokenizer_path = Path("tokenizer_{0}.json".format(language))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token=UNK_TOKEN))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=[UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(dataset, language), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

def get_dataset():
    dataset = load_dataset(DATASET_NAME, SOURCE_LANGUAGE + "-" + TARGET_LANGUAGE, split="train")
    dataset = dataset.shuffle().select(range(int(len(dataset)/5)))

    src_tokenizer = get_tokenizer(dataset, SOURCE_LANGUAGE)
    trg_tokenizer = get_tokenizer(dataset, TARGET_LANGUAGE)

    max_length = SEQUENCE_LENGTH - 2

    def is_valid_length(example):
        src_tokens = src_tokenizer.encode(example['translation'][SOURCE_LANGUAGE]).ids
        trg_tokens = trg_tokenizer.encode(example['translation'][TARGET_LANGUAGE]).ids
        return len(src_tokens) <= max_length and len(trg_tokens) <= max_length

    dataset = dataset.filter(is_valid_length)
    print(f"Dataset filtred: {len(dataset)} examples left")

    train_size = int(0.8 * len(dataset))
    validation_size = len(dataset) - train_size

    train_dataset, valisation_dataset = random_split(dataset, [train_size, validation_size])

    train_dataset = EnglishToFrenchDataset(train_dataset, src_tokenizer, trg_tokenizer, SOURCE_LANGUAGE, TARGET_LANGUAGE, SEQUENCE_LENGTH)
    validation_dataset = EnglishToFrenchDataset(valisation_dataset, src_tokenizer, trg_tokenizer, SOURCE_LANGUAGE, TARGET_LANGUAGE, SEQUENCE_LENGTH)

    # Find the maximum length of each sentence in the source and target sentence
    src_max_length = 0
    trg_max_length = 0

    for item in dataset:
        src_ids = src_tokenizer.encode(item['translation'][SOURCE_LANGUAGE]).ids
        trg_ids = trg_tokenizer.encode(item['translation'][TARGET_LANGUAGE]).ids
        src_max_length = max(src_max_length, len(src_ids))
        trg_max_length = max(trg_max_length, len(trg_ids))

    print(f'Max length of source sentence: {src_max_length}')
    print(f'Max length of target sentence: {trg_max_length}')


    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(validation_dataset, batch_size=1, shuffle=True) # Process the sentences one by one

    return train_dataloader, val_dataloader, src_tokenizer, trg_tokenizer

In [14]:
def get_transformer(src_vocab_size, trg_vocab_size):
    transformer = Transformer(model_dimension=MODEL_DIMENSION,
                              inner_layer_dimension=MODEL_INNER_LAYER_DIMENSION,
                              number_of_layers=MODEL_NUMBER_OF_LAYERS,
                              number_of_heads=MODEL_NUMBER_OF_HEADS,
                              src_vocabulary_size=src_vocab_size,
                              trg_vocabulary_size=trg_vocab_size
                            )
    return transformer


def greedy_decode(model, source, source_mask, src_tokenizer, trg_tokenizer, max_len, device):
    sos_idx = trg_tokenizer.token_to_id('[SOS]')
    eos_idx = trg_tokenizer.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)

    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, decoder_input, source_mask, decoder_mask)

        # get next token
        prob = model.linear_projection(out[:, -1])
        prob = model.softmax(prob)
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)


def run_validation(model, validation_ds, src_tokenizer, trg_tokenizer, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

            # check that the batch size is 1
            assert encoder_input.size(
                0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, src_tokenizer, trg_tokenizer, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["trg_text"][0]
            model_out_text = trg_tokenizer.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break

    if writer:
        # Evaluate the character error rate
        # Compute the char error rate
        metric = torchmetrics.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        # Compute the BLEU metric
        metric = torchmetrics.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()


def learning_rate(step):
    if step == 0:
        step = 1
    return MODEL_DIMENSION ** (-0.5) * min(step ** (-0.5), step * WARMUP_STEPS ** (-1.5))


def train_model():
    # Define the device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    if (device == 'cuda'):
        print(f"Device name: {torch.cuda.get_device_name(device.index)}")
        print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
    device = torch.device(device)
    torch.cuda.empty_cache()

    # Make sure the weights folder exists
    Path(f"{DATASET_NAME}_{MODEL_FOLDER}").mkdir(parents=True, exist_ok=True)

    train_dataloader, val_dataloader, src_tokenizer, trg_tokenizer = get_dataset()
    model = get_transformer(src_tokenizer.get_vocab_size(), trg_tokenizer.get_vocab_size()).to(device)

    # Tensorboard
    writer = SummaryWriter(EXPERIMENT_FOLDER)

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(BETA1, BETA2), eps=EPSILON)
    #lr_scheduler = LambdaLR(optimizer=optimizer, lr_lambda=lambda step: learning_rate(step))

    # If the user specified a model to preload before training, load it
    initial_epoch = 1
    global_step = 0
    preload = MODEL_PRELOAD
    model_filename = latest_weights_file_path() if preload == 'latest' else get_weights_file_path(preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    loss_function = nn.CrossEntropyLoss(ignore_index=trg_tokenizer.token_to_id('[PAD]'), label_smoothing=MODEL_LABEL_SMOOTHING_VALUE).to(device)

    for epoch in range(initial_epoch, NUMBER_OF_EPOCHS + 1):
        torch.cuda.empty_cache()
        model.train()
        batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:

            encoder_input = batch['encoder_input'].to(device) # (B, seq_len)
            decoder_input = batch['decoder_input'].to(device) # (B, seq_len)
            encoder_mask = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
            decoder_mask = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)

            # Run the tensors through the encoder, decoder and the projection layer
            model_output = model(encoder_input, decoder_input, encoder_mask, decoder_mask)

            # Compare the output with the label
            label = batch['label'].to(device) # (B, seq_len)

            # Compute the loss using a simple cross entropy
            loss = loss_function(model_output.view(-1, trg_tokenizer.get_vocab_size()), label.view(-1))
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            # Log the loss
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Backpropagate the loss
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Update the weights
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            #lr_scheduler.step()

            global_step += 1

        # Run validation at the end of every epoch
        run_validation(model, val_dataloader, src_tokenizer, trg_tokenizer, SEQUENCE_LENGTH, device, lambda msg: batch_iterator.write(msg), global_step, writer)

        # Save the model at the end of every epoch
        model_filename = get_weights_file_path(f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

In [15]:
warnings.filterwarnings("ignore")
train_model()

Using device: cuda
Device name: Tesla T4
Device memory: 14.74127197265625 GB


Filter:   0%|          | 0/25417 [00:00<?, ? examples/s]

Dataset filtred: 25417 examples left
Max length of source sentence: 355
Max length of target sentence: 363
No model to preload, starting from scratch


Processing Epoch 01: 100%|██████████| 5084/5084 [07:55<00:00, 10.68it/s, loss=4.341]


--------------------------------------------------------------------------------
    SOURCE: "Then you use me, and yet do not trust me!" I cried with some bitterness.
    TARGET: – Ainsi vous vous servez de moi, et pourtant vous ne vous fiez pas à moi ! m’écriai-je avec amertume.
 PREDICTED: Alors je ne me me me me me dit que vous me faire , et je ne me pas .
--------------------------------------------------------------------------------
    SOURCE: Anne of Austria lowered her head, allowed the torrent to flow on without replying, hoping that it would cease of itself; but this was not what Louis XIII meant. Louis XIII wanted a discussion from which some light or other might break, convinced as he was that the cardinal had some afterthought and was preparing for him one of those terrible surprises which his Eminence was so skillful in getting up.
    TARGET: Anne d'Autriche baissa la tête, laissa s'écouler le torrent sans répondre et espérant qu'il finirait par s'arrêter; mais ce n'éta

...

--------------------------------------------------------------------------------
    SOURCE: De temps en temps, il est vrai, il se présente un cas un peu plus compliqué.
    TARGET: Now and again a case turns up which is a little more complex.
 PREDICTED: " You ' t ," he said , " I ' s a in the , " I should have a .
--------------------------------------------------------------------------------
    SOURCE: The captain adjusted Jehan's head upon an inclined plane of cabbage−stumps, and on the very instant, the scholar fell to snoring in a magnificent bass.
    TARGET: Le capitaine arrangea la tête de Jehan sur un plan incliné de trognons de choux, et à l’instant même l’écolier se mit à ronfler avec une basse-taille magnifique.
 PREDICTED: Le capitaine , fit un instant sur le capitaine , et tomba sur la tête de la tête , sur la tête de la tête , et se faire une tête en fit signe de se faire .
--------------------------------------------------------------------------------


Processing Epoch 09: 100%|██████████| 5084/5084 [07:51<00:00, 10.78it/s, loss=4.108]


--------------------------------------------------------------------------------
    SOURCE: The next day, as she was getting up, she saw the clerk on the Place.
    TARGET: Le lendemain, à son réveil, elle aperçut le clerc sur la place.
 PREDICTED: Le lendemain , elle vit le lendemain , comme sur le lendemain , elle vit sur la place .
--------------------------------------------------------------------------------
    SOURCE: 'Ah, my dear Julien,' said M. de Renal, laughing in the most insincere manner, 'the whole day, if you wish, the whole of tomorrow, my worthy friend.
    TARGET: – Eh, mon cher Julien ! dit M. de Rênal en riant de l’air le plus faux, toute la journée, si vous voulez, toute celle de demain, mon bon ami.
 PREDICTED: – Ah ! mon cher ami , dit Julien en se , si vous le plus grand monde , vous , vous avez tout ce jour , vous vous , vous tout est à fait de mon cher ami .
--------------------------------------------------------------------------------


Processing Epoch 10: 100%|██████████| 5084/5084 [07:52<00:00, 10.77it/s, loss=5.002]


--------------------------------------------------------------------------------
    SOURCE: Yet in spite of all this seduction Milady might fail--for Felton was forewarned, and that against the least chance.
    TARGET: Et cependant, malgré toute cette séduction, Milady pouvait échouer, car Felton était prévenu, et cela contre le moindre hasard.
 PREDICTED: Mais , en avant tout , Milady était le moins contre tout , et Milady se contre tout .
--------------------------------------------------------------------------------
    SOURCE: By way of decoration for the apartment, hanging to a nail in the middle of the wall, whose green paint scaled off from the effects of the saltpetre, was a crayon head of Minerva in gold frame, underneath which was written in Gothic letters "To dear Papa."
    TARGET: Il y avait, pour décorer l’appartement, accrochée à un clou, au milieu du mur dont la peinture verte s’écaillait sous le salpêtre, une tête de Minerve au crayon noir, encadrée de dorure, et qu

In [13]:
def test_overfitting():
    """Test si le modèle peut overfit sur un seul batch (il devrait le faire facilement)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)
    train_dataloader, val_dataloader, src_tokenizer, trg_tokenizer = get_dataset()
    model = get_transformer(src_tokenizer.get_vocab_size(), trg_tokenizer.get_vocab_size()).to(device)
    # Prenez 1 seul batch
    single_batch = next(iter(train_dataloader))

    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    print("Testing overfitting on single batch...")
    for step in range(100):
        encoder_input = single_batch['encoder_input'].to(device)
        decoder_input = single_batch['decoder_input'].to(device)
        encoder_mask = single_batch['encoder_mask'].to(device)
        decoder_mask = single_batch['decoder_mask'].to(device)
        label = single_batch['label'].to(device)

        model_output = model(encoder_input, decoder_input, encoder_mask, decoder_mask)
        loss = nn.CrossEntropyLoss(ignore_index=trg_tokenizer.token_to_id('[PAD]'))(
            model_output.view(-1, trg_tokenizer.get_vocab_size()),
            label.view(-1)
        )

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % 20 == 0:
            print(f"Step {step}: Loss = {loss.item():.4f}")

    if loss.item() < 0.1:
        print("✅ Model CAN overfit - the architecture is working")
    else:
        print("❌ Model CANNOT overfit - there's a fundamental problem")

test_overfitting()

Using device: cuda


Filter:   0%|          | 0/25417 [00:00<?, ? examples/s]

Dataset filtred: 25417 examples left
Max length of source sentence: 361
Max length of target sentence: 350
Testing overfitting on single batch...
Step 0: Loss = 10.4606
Step 20: Loss = 0.2373
Step 40: Loss = 0.0143
Step 60: Loss = 0.0067
Step 80: Loss = 0.0050
✅ Model CAN overfit - the architecture is working
